In [1]:
import numpy as np
from astropy.io import fits
import json


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [2]:
hdul = fits.open("gll_psc_v30.fit")
data = hdul[1].data
cols = hdul[1].columns.names
mask = (data["CLASS1"].rfind("MSP") >= 0) | (data["CLASS1"].rfind("PSR") >= 0)
data = data[mask]

bins = np.array(
    [[hdul[5].data[i][0], hdul[5].data[i][1]] for i in range(len(hdul[5].data))]
)
bins = np.array([np.unique(bins[:, 0]), np.unique(bins[:, 1])]).T

cols = "Source_Name;RAJ2000;DEJ2000;GLON;GLAT;Signif_Avg;Energy_Flux100;Unc_Energy_Flux100;SpectrumType;PL_Flux_Density;Unc_PL_Flux_Density;PL_Index;Unc_PL_Index;PLEC_Flux_Density;Unc_PLEC_Flux_Density;PLEC_IndexS;Unc_PLEC_IndexS;PLEC_ExpfactorS;Unc_PLEC_ExpfactorS;PLEC_Exp_Index;Unc_PLEC_Exp_Index;PLEC_SigCurv;PLEC_EPeak;Unc_PLEC_EPeak;Flux_Band;Unc_Flux_Band;nuFnu_Band;ASSOC_TEV;Flags"
units = ";deg;deg;deg;deg;;erg cm-2 s-1;erg cm-2 s-1;;cm-2 MeV-1 ph s-1;cm-2 MeV-1 ph s-1;;;cm-2 MeV-1 ph s-1;cm-2 MeV-1 ph s-1;;;;;;;;MeV;MeV;cm-2 s-1;cm-2 s-1;erg cm-2 s-1;;"

data_ = {k: (np.array(data[k]), u) for k, u in zip(cols.split(";"), units.split(";"))}
data_["Name"] = (data["ASSOC1"].replace("PSR ", "").strip(), "")
data = data_
data["Band"] = (bins, "MeV")

data = [
    {k: [v[0][i], v[1]] if k != "Band" else [bins, "MeV"] for k, v in data.items()}
    for i in range(len(data["Source_Name"][0]))
]

In [191]:
json.dump(data, open("fermi.json", "w"), cls=NpEncoder)

In [192]:
cols = ["Name", "P0", "dP0", "P1", "dP1", "Dist", "Edot", "Bsurf"]
data = np.loadtxt(
    "psrcat.csv",
    delimiter=";",
    dtype=str,
    usecols=(3, 5, 6, 8, 9, 11, 12, 13),
    converters={3: lambda s: s.strip()},
)[2:]
data = data[(data[:, 1] != "*") & (data[:, 3] != "*")]
data = {
    k: (np.array(data[:, i]), u)
    for i, (k, u) in enumerate(zip(cols, ["", "s", "s", "", "", "kpc", "erg/s", "G"]))
}
data = {
    k: (
        np.array(
            [np.nan if v == "*" else v for v in data[k][0]],
            dtype=float if k != "Name" else str,
        ),
        data[k][1],
    )
    for k in cols
}

# invert data to array of objects
data = [
    {k: [v[0][i], v[1]] for k, v in data.items()} for i in range(len(data["Name"][0]))
]

In [193]:
json.dump(data, open("atnf.json", "w"), cls=NpEncoder)